# Lakebase Database Instance Setup
Create Databricks Lakebase (PostgreSQL) instance, catalog, and tables


In [3]:
# Force reload of data_store_variables
import sys
import importlib

# Remove from cache if exists
if 'data_store_variables' in sys.modules:
    del sys.modules['data_store_variables']


print("\n✅ Variables forcefully reloaded from data_store_variables.ipynb")


✅ Variables forcefully reloaded from data_store_variables.ipynb


In [4]:
%run ./data_store_variables


📋 CONFIGURATION VARIABLES

🏗️  PROJECT:
   PROJECT_NAME: telecom_iot

📚 DATA CONFIGURATION:
   CATALOG_NAME: kunal
   SCHEMA_NAME: Telcom
   VOLUME_NAME: raw_data
   VOLUME_PATH: /Volumes/kunal/Telcom/raw_data
   TABLE_NAME: kunal.Telcom.iot_data_synched_cont

👤 USER:
   DB_USER: kunal.gaurav
   DB_USER_EMAIL: kunal.gaurav@databricks.com
   DB_USER_CLEAN: kunal-gaurav

🖥️  APPLICATION:
   APP_NAME: iot_dashboard
   DATA_REFRESH_INTERVAL: 500 minutes

🗄️  LAKEBASE CONFIGURATION:
   INSTANCE_NAME: kunal-gaurav-lakebase-instance
   INSTANCE_CAPACITY: CU_1
   LAKEBASE_CATALOG_NAME: pg_lakebase_kunal-gaurav
   SYNCED_TABLE_NAME: pg_lakebase_kunal-gaurav.Telcom.iot_data_synced
   PG_TABLE_NAME: kunal.Telcom.iot_metadata
✅ Catalog 'kunal' created or already exists
✅ Schema 'kunal.Telcom' created or already exists
✅ Table 'kunal.Telcom.iot_data_synched_cont' created or already exists

🎉 Database setup complete!
✅ Volume 'kunal.Telcom.raw_data' created or already exists
   Path: /Volumes/kunal/

In [5]:
%pip install databricks-sdk

from databricks.sdk import WorkspaceClient
from databricks.sdk.service.database import (
    DatabaseInstance,
    DatabaseCatalog,
    DatabaseTable,
    SyncedDatabaseTable)
from databricks.sdk.service.database import (
    DatabaseInstance,
    DatabaseCatalog,
    DatabaseTable,
    SyncedDatabaseTable,
    SyncedTableSpec,
    NewPipelineSpec,
    SyncedTableSchedulingPolicy  # ✅ Add this import
)
import time

print("✅ Databricks SDK imported")


Note: you may need to restart the kernel to use updated packages.
✅ Databricks SDK imported


In [6]:
# Initialize Databricks Workspace Client
w = WorkspaceClient()

print(f"✅ Connected to workspace: {w.config.host}")
print(f"   User: {w.current_user.me().user_name}")


✅ Connected to workspace: https://e2-demo-field-eng.cloud.databricks.com
   User: kunal.gaurav@databricks.com


In [7]:
# All variables are loaded from data_store_variables.ipynb



print(f"✅ Configuration loaded:")
print(f"   Project: {PROJECT_NAME}")
print(f"   Instance: {INSTANCE_NAME}")
print(f"   Capacity: {INSTANCE_CAPACITY}")
print(f"   SYNCED_TABLE_NAME: {SYNCED_TABLE_NAME}")
print(f"LAKEBASE_CATALOG_NAME: {LAKEBASE_CATALOG_NAME}")

✅ Configuration loaded:
   Project: telecom_iot
   Instance: kunal-gaurav-lakebase-instance
   Capacity: CU_1
   SYNCED_TABLE_NAME: pg_lakebase_kunal-gaurav.Telcom.iot_data_synced
LAKEBASE_CATALOG_NAME: pg_lakebase_kunal-gaurav


In [10]:
# Check if database instance exists
def get_database_instance_by_name(instance_name):
    """Check if database instance exists"""
    for instance in w.database.list_database_instances():
        if instance.name == instance_name:
            return instance
    return None

# Check if instance exists
existing_instance = get_database_instance_by_name(INSTANCE_NAME)

if existing_instance:
    print(f"✅ Database Instance '{INSTANCE_NAME}' already exists")
    print(f"   Instance ID: {existing_instance.uid}")
    print(f"   Capacity: {existing_instance.capacity}")
    print(f"   State: {existing_instance.state}")
    instance = existing_instance
else:
    print(f"🔄 Creating Database Instance: {INSTANCE_NAME}...")
    print(f"   Capacity: {INSTANCE_CAPACITY}")
    print(f"   This may take 5-10 minutes...")
    
    # Create database instance
    instance = w.database.create_database_instance_and_wait(
        database_instance=DatabaseInstance(
            name=INSTANCE_NAME,
            capacity=INSTANCE_CAPACITY
        )
    )
    
    print(f"✅ Database Instance created successfully!")
    print(f"   Instance Name: {instance.name}")
    print(f"   Instance UID: {instance.uid}")
    print(f"   Capacity: {instance.capacity}")
    print(f"   State: {instance.state}")

# Save instance name for later use
LAKEBASE_INSTANCE_NAME = instance.name
print(f"\n💾 Instance Name: {LAKEBASE_INSTANCE_NAME}")


✅ Database Instance 'kunal-gaurav-lakebase-instance' already exists
   Instance ID: f60d62f1-e44a-43c7-813f-58138e0552fd
   Capacity: CU_1
   State: DatabaseInstanceState.AVAILABLE

💾 Instance Name: kunal-gaurav-lakebase-instance


In [11]:
# Create or get database catalog
# Use LAKEBASE_CATALOG_NAME from data_store_variables

print(f"🔄 Setting up Database Catalog: {LAKEBASE_CATALOG_NAME}...")

try:
    # Try to create database catalog
    db_catalog = w.database.create_database_catalog(
        catalog=DatabaseCatalog(
            name=LAKEBASE_CATALOG_NAME,
            database_instance_name=LAKEBASE_INSTANCE_NAME,
            database_name=LAKEBASE_CATALOG_NAME,
            create_database_if_not_exists=True
        )
    )
    
    print(f"✅ Database Catalog created!")
    print(f"   Catalog Name: {db_catalog.name}")
    print(f"   Database Name: {db_catalog.database_name}")
    
except Exception as e:
    error_msg = str(e)
    
    # Check if it's an "already exists" error
    if "already exists" in error_msg.lower():
        print(f"✅ Database Catalog '{LAKEBASE_CATALOG_NAME}' already exists")
        print(f"   Using existing catalog")
        # Set db_catalog to None or a placeholder since we can't retrieve it
        db_catalog = None
    else:
        # Re-raise if it's a different error
        print(f"❌ Unexpected error: {e}")
        raise

print(f"\n💾 Lakebase Catalog: {LAKEBASE_CATALOG_NAME}")
print(f"   Instance: {LAKEBASE_INSTANCE_NAME}")

🔄 Setting up Database Catalog: pg_lakebase_kunal-gaurav...
✅ Database Catalog 'pg_lakebase_kunal-gaurav' already exists
   Using existing catalog

💾 Lakebase Catalog: pg_lakebase_kunal-gaurav
   Instance: kunal-gaurav-lakebase-instance


In [39]:
# Create a synced table (syncs from UC Delta to PostgreSQL)
# SYNCED_TABLE_NAME is already loaded from data_store_variables



print(f"🔄 Creating Synced Database Table: {SYNCED_TABLE_NAME}...")

try:
    # Define the pipeline specification for the sync
    pipeline_spec = NewPipelineSpec(
        # Location for pipeline checkpoints and logs
        # Must have write permissions to this catalog/schema
        storage_catalog=LAKEBASE_CATALOG_NAME,
        storage_schema=SCHEMA_NAME
    )
    
    # Create the synced table spec
    synced_spec = SyncedTableSpec(
        source_table_full_name=TABLE_NAME,  # Source Delta table
      
        primary_key_columns=["tower_id", "timestamp"],  # Primary keys for upserts
        create_database_objects_if_missing=True,  # Auto-create PG objects
        scheduling_policy=SyncedTableSchedulingPolicy.CONTINUOUS,
        new_pipeline_spec= pipeline_spec   # Options: CONTINUOUS, SNAPSHOT, TRIGGERED
    )
    
    # Create the synced database table
    synced_table = w.database.create_synced_database_table(
        synced_table=SyncedDatabaseTable(
            name=SYNCED_TABLE_NAME,  # UC catalog.schema.table name
            database_instance_name=LAKEBASE_INSTANCE_NAME, 
            logical_database_name=LAKEBASE_CATALOG_NAME, # Lakebase instance
            spec=synced_spec# ✅ Use spec parameter
        )
        
    )
    
    print(f"✅ Synced Table created!")
    print(f"   Table Name: {synced_table.name}")
    print(f"   Source: {TABLE_NAME}")
    print(f"   Target Instance: {LAKEBASE_INSTANCE_NAME}")
    print(f"   Scheduling: CONTINUOUS")
except Exception as e:
    print(f"⚠️  Error: {e}")
    print(f"   Make sure source table '{TABLE_NAME}' exists and has data")


🔄 Creating Synced Database Table: kunal.Telcom.iot_data_synced...
✅ Synced Table created!
   Table Name: kunal.telcom.iot_data_synced
   Source: kunal.Telcom.iot_data_synched_cont
   Target Instance: kunal-gaurav-lakebase-instance
   Scheduling: CONTINUOUS


Working with lakebase data

In [ ]:
# Generate credentials for the Lakebase instance
instance = w.database.get_database_instance(name=LAKEBASE_INSTANCE_NAME)
credential = w.database.generate_database_credential(
    instance_names=[LAKEBASE_INSTANCE_NAME]
)
host = instance.read_write_dns
port = 5432
dbname = "databricks_postgres"
user = DB_USER_EMAIL
password = credential.token

print(host)
print(port)
print(dbname)
print(user)


In [43]:
# Generate app.yml with connection values
print("📝 Generating app.yml with connection values...")

# Get instance details
instance = w.database.get_database_instance(name=LAKEBASE_INSTANCE_NAME)
host = instance.read_write_dns
port = 5432

app_yml_content = f"""command:
  - "python"
  - "app.py"

env:
  # PostgreSQL Database (from DatabaseCatalog)
  - name: PGDATABASE
    value: "{LAKEBASE_CATALOG_NAME}"
  
  # Database User
  - name: PGUSER
    value: "{DB_USER_EMAIL}"
  
  # PostgreSQL Host (from Lakebase instance)
  - name: PGHOST
    value: "{host}"
  
  # PostgreSQL Port
  - name: PGPORT
    value: "{port}"
  
  # SSL Mode (required for Databricks)
  - name: PGSSLMODE
    value: "require"
  
  # Application Name
  - name: PGAPPNAME
    value: "{APP_NAME}"
  
  # Table Configuration
  - name: SCHEMA_NAME
    value: "{CATALOG_NAME}.{SCHEMA_NAME}"
  
  - name: TABLE_NAME
    value: "{SYNCED_TABLE_NAME.split('.')[-1]}"
"""

# Write to file
app_yml_path = "../lakebase_apps/app.yml"
with open(app_yml_path, 'w') as f:
    f.write(app_yml_content)

print(f"✅ app.yml generated successfully!")
print(f"   Path: {app_yml_path}")
print(f"   PGHOST: {host}")
print(f"   PGDATABASE: {LAKEBASE_CATALOG_NAME}")
print(f"   TABLE: {SYNCED_TABLE_NAME}")

📝 Generating app.yml with connection values...
✅ app.yml generated successfully!
   Path: ../lakebase_apps/app.yml
   PGHOST: instance-f60d62f1-e44a-43c7-813f-58138e0552fd.database.cloud.databricks.com
   PGDATABASE: pg_lakebase_kunal-gaurav
   TABLE: kunal.Telcom.iot_data_synced


In [1]:
%pip install psycopg

Note: you may need to restart the kernel to use updated packages.


In [19]:

# Create PostgreSQL Role for Databricks App Service Principal
import psycopg

print("🔐 Setting up PostgreSQL Role for App Service Principal...")

# Get connection
instance = w.database.get_database_instance(name=LAKEBASE_INSTANCE_NAME)
credential = w.database.generate_database_credential(instance_names=[LAKEBASE_INSTANCE_NAME])

# Connect as admin user
conn = psycopg.connect(
    host=instance.read_write_dns,
    port=5432,
    dbname=LAKEBASE_CATALOG_NAME,
    user=DB_USER_EMAIL,
    password=credential.token,
    sslmode="require"
)

conn.autocommit = True
cur = conn.cursor()

# Your app's service principal ID
APP_SERVICE_PRINCIPAL = "1e2bd8c8-1a42-4d68-af82-6d83250ad4c2"

try:
    # Check if role exists
    cur.execute(f"""
        SELECT 1 FROM pg_roles WHERE rolname = '{APP_SERVICE_PRINCIPAL}'
    """)
    
    if cur.fetchone():
        print(f"   Role already exists")
    else:
        # Create the role for the service principal
        cur.execute(f'CREATE ROLE "{APP_SERVICE_PRINCIPAL}" WITH LOGIN')
        print(f"✅ Created PostgreSQL role for service principal")
    
    # Grant necessary permissions
    cur.execute(f'GRANT CONNECT ON DATABASE "{LAKEBASE_CATALOG_NAME}" TO "{APP_SERVICE_PRINCIPAL}"')
    cur.execute(f'GRANT USAGE ON SCHEMA telcom TO "{APP_SERVICE_PRINCIPAL}"')
    cur.execute(f'GRANT SELECT ON ALL TABLES IN SCHEMA telcom TO "{APP_SERVICE_PRINCIPAL}"')
    
    print(f"✅ Granted permissions to service principal")
    print(f"\n🎉 Service principal role configured successfully!")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
finally:
    cur.close()
    conn.close()

🔐 Setting up PostgreSQL Role for App Service Principal...
✅ Created PostgreSQL role for service principal
✅ Granted permissions to service principal

🎉 Service principal role configured successfully!
